## <center>CSE 546: Reinforcement Learning</center>
### <center>Prof. Alina Vereshchaka</center>
<!-- ### <center>Fall 2022</center> -->

Welcome to the Assignment 2, Part 1: Introduction to Deep Reinforcement Learning and Neural Networks! The goal of this assignment is to make you comfortable with the application of different Neural Network structures depending on how the Reinforcement Learning environment is set up.

In [3]:
# Imports
from environment import WumpusWorldEnvironment

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


c:\Users\abhil\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Environment

We will be working with an implementation of the Wumpus World environment. The environment comes from the book "Artificial Intelligence: A Modern Approach" by Stuart J. Russell and Peter Norvig. 

### ENVIRONMENT DETAILS:

The environment is a 6 x 6 grid world containing a total of 36 grid blocks. 

#### ENVIRONMENT OBJECTS:
The environment consists of the following objects:

1. **Agent** - The agent starts in the grid block at the bottom left corner whose co-ordinates are [0, 0]. The goal of our agent is to collect the Gold while avoiding the Wumpus and the pits. 

2. **Wumpus** - The monster which would eat the agent if they are in the same grid block.

3. **Pit** - The agent must avoid falling into the pits. 

4. **Gold** - The agent must collect the Gold.

5. **Breeze** - Breeze surrounds the Pits and warn the agent of a Pit in an adjacent grid block.

6. **Stench** - Stench surrounds the Wumpus and warns the agent of the Wumpus in an adjacent grid block.

#### ENVIRONMENT OBSERVATIONS:

Our implementation of the environment provides you with four different types of observations:

1. **Integer** - Integer in the range [0 - 35]. This represents the grid block the agent is in. E.g., if the agent is in the bottom left grid block (starting position) the observation would be 0, if the agent is in the grid block containing the Gold the observation would be 34, if the agent is in the top right grid block the observation would be 35.

2. **Vector** - 

    **2.1.** A vector of length 2 representing the agent co-ordinates. The first entry represents the x co-ordinate and the second entry represets the y co-ordinate. E.g., if the agent is in the bottom left grid block (starting position) the observation would be [0, 0], if the agent is in the grid block containing the Gold the observation would be [4, 5], if the agent is in the top right grid block the observation would be [5, 5].
    
    **2.2.** A vector of length 36 representing the one-hot encoding of the integer observation (refer type 1 above). E.g., if the agent is in the bottom left grid block (starting position) the observation would be [1, 0, ..., 0, 0], if the agent is in the grid block containing the Gold the observation would be [0, 0, ..., 1, 0], if the agent is in the top right grid block the observation would be [0, 0, ..., 0, 1].


3. **Image** - Image render of the environment returned as an NumPy array. The image size is 84 * 84 (same size used in the DQN paper). E.g., if the agent is in the bottom right grid block the observation is:

    Observation: (84 * 84)

     [[255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     ...

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]]

    Observation type: <class 'numpy.ndarray'>

    Observation Shape: (84, 84)

    Visually, it looks like:
    <img src="./images/environment_render.png" width="500" height="500">
    

4. **Float** - Float in the range [0 - $\infty$] representing the time elapsed in seconds. 

#### ENVIRONMENT ACTIONS:

Our implementation of the environment provides you with three different types of actions:

1. **Discrete** - Integer in the range [0 - 3] representing the four actions possible in the environment as follows: 0 - Right 1 - Left 2 - Up 3 - Down.

2. **Multi-Discrete** - Array of length four where each element takes binary values 0 or 1. Array elements represent if we take a particular action. Array element with index 0 corresponds to the right action, index 1 corresponds to the left action, index 2 corresponds to the up action, and index 3 corresponds to the down action. E.g., 
   action = [1, 0, 0, 0] would result in the agent moving right.
   action = [1, 0, 1, 0] would result in the agent moving right and up.
   action = [0, 1, 0, 1] would result in the agent moving left and down.

3. **Continuous** - Float in the range [-1, 1] determining whether the agent will go left, right, up, or down as follows:

    if -1 <= action <= -0.5:
        Go Right.
    elif -0.5 < action <= 0:
        Go Left.
    elif 0 < action <= 0.5:
        Go Up.
    elif 0.5 < action <= 1:
        Go Down.
        
### YOUR TASK IS TO USE A NEURAL NETWORK TO WORK WITH ALL FOUR TYPES OF OBSERVATIONS AND ALL THREE TYPES OF  ACTIONS.

<img src="./images/wumpus_world_environment.jpg" width="600" height="600">

# START COMPLETING YOUR ASSIGNMENT HERE

## Observation Type - Integer, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is an integer (refer to environment observations type 1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [4]:
def getAction(index:int):
    if index == 0:
        return "RIGHT"
    if index == 1:
        return "LEFT"   
    if index == 2:
        return "UP"
    if index == 3:
        return "DOWN"               

In [5]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation = environment.reset()

print(observation)
# BEGIN_YOUR_CODE

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np



class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 64)  # 5*5 from image dimension
        self.fc2 = nn.Linear(64, 4)
  
    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

   


net = Net()
observation_tensor = torch.tensor(observation, dtype=torch.float32)
observation_tensor = observation_tensor.view(-1, 1)
output = net(observation_tensor)
print(output)
output_np_array = output.detach().numpy()
action_value = np.argmax(output_np_array)
action = getAction(action_value)
print("Action value is", action_value)
print("Action is",action)
print ("New observation is", environment.step(action_value))

# END_YOUR_CODE

0
tensor([[ 0.5462, -0.0887,  0.6681,  0.1092]], grad_fn=<AddmmBackward0>)
Action value is 2
Action is UP
New observation is (6, None, None, {})


## Observation Type - Vector (2.1), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 2 (refer to environment observations type 2.1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_2_64_4.png">

In [6]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='vector', action_type='discrete')

observation = environment.reset()
print('Current observation is', observation)

# BEGIN_YOUR_CODE
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 64)  # 5*5 from image dimension
        self.fc2 = nn.Linear(64, 4)
        

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

 


net = Net()
observation_tensor = torch.tensor(observation, dtype=torch.float32)
observation_tensor = observation_tensor.view(-1, 2)
output = net(observation_tensor)
print(output)

output_np_array = output.detach().numpy()
action_value = np.argmax(output_np_array)
action = getAction(action_value)
print("Action value is", action_value)
print("Action is",action)
print ("New observation is", environment.step(action_value))
# END_YOUR_CODE

Current observation is [0 0]
tensor([[ 0.2176, -0.1916,  0.0032, -0.3261]], grad_fn=<AddmmBackward0>)
Action value is 0
Action is RIGHT
New observation is (array([1, 0]), None, None, {})


## Observation Type - Vector (2.2), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4.png">

In [7]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(
    observation_type='vector', action_type='discrete')
observation = environment.reset()

# BEGIN_YOUR_CODE


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(36, 64)  # 5*5 from image dimension
        self.fc2 = nn.Linear(64, 4)

    def forward(self, x):
        # flatten all dimensions except the batch dimension
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

environment_height = 6
environment_width = 6
coordinates_state_mapping = {}

for i in range(environment_height):
    for j in range(environment_width):
        coordinates_state_mapping[f'{np.asarray([j, i])}'] = i * \
            environment_width + j

observation_state = coordinates_state_mapping[f'{observation}']
print("Current observation is", observation_state)

observation_vector = np.zeros(36)
observation_vector[observation_state] = 1
observation_tensor = torch.tensor(observation_vector, dtype=torch.float32)
observation_tensor = observation_tensor.view(-1, 36)
output = (net(observation_tensor))

print(output)

output_np_array = output.detach().numpy()
action_value = np.argmax(output_np_array)
action = getAction(action_value)
print("Action value is", action_value)
print("Action is", action)
print("New observation is", environment.step(action_value))


# END_YOUR_CODE


Current observation is 0
tensor([[-0.0287,  0.1351,  0.0962, -0.0751]], grad_fn=<AddmmBackward0>)
Action value is 1
Action is LEFT
New observation is (array([0, 0]), None, None, {})


## Observation Type - Image, Action Type - Discrete

The part of the assignment requires you to create a convolutional neural network with one convolutional layer having 128 filters of size 3 x 3, one hidden layer having 64 neurons, and the output layer having 4 neurons. The input to the neural network is an image of size 84 * 84 (refer to environment observations type 3). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/convolutional_neural_network_84x84_128_64_4.png">

In [8]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='image', action_type='discrete')
observation = environment.reset()


# BEGIN_YOUR_CODE
from torchvision import transforms

from PIL import Image
import numpy as np
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 128, 3)
        self.conv1 = nn.Conv2d(1, 128, 3)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(128 * 41 * 41 , 64)
        self.fc2 = nn.Linear(64, 4)
        # # self.conv2 = nn.Conv2d(6, 16, 5)
        # # an affine operation: y = Wx + b
        # self.fc1 = nn.Linear(128 * 84 * 84, 64)  # 5*5 from image dimension
        # self.fc2 = nn.Linear(64, 4)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x




net = Net()
convert_tensor = transforms.ToTensor()
observation_tensor = convert_tensor(observation)
observation_tensor = observation_tensor[None, :, :, : ]
output = net(observation_tensor)
print(output)

output_np_array = output.detach().numpy()
action_value = np.argmax(output_np_array)
action = getAction(action_value)
print("Action value is", action_value)
print("Action is", action)
print("New observation is", environment.step(action_value))


# END_YOUR_CODE

tensor([[-0.0625,  0.2546,  0.1643,  0.1117]], grad_fn=<AddmmBackward0>)
Action value is 1
Action is LEFT
New observation is (array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), None, None, {})


## Observation Type - Float, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a float (refer to environment observations type 4). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [9]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='float', action_type='discrete')
observation = environment.reset()
print(observation)
import numpy as np
# BEGIN_YOUR_CODE

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np



class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 64)  # 5*5 from image dimension
        self.fc2 = nn.Linear(64, 4)
  
    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

   


net = Net()
observation_tensor = torch.tensor(observation, dtype=torch.float32)
observation_tensor = observation_tensor.view(-1, 1)
output = net(observation_tensor)
print(output)

output_np_array = output.detach().numpy()
action_value = np.argmax(output_np_array)
action = getAction(action_value)
print("Action value is", action_value)
print("Action is", action)
print("New observation is", environment.step(action_value))


# # END_YOUR_CODE


0.0
tensor([[-0.2273, -0.2290, -0.0444, -0.0679]], grad_fn=<AddmmBackward0>)
Action value is 2
Action is UP
New observation is (0.0070226192474365234, None, None, {})


## Observation Type - Vector (2.2), Action Type - Multi-Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array representing the probability of choosing the actions. (If the value of the array element is >=0.5 you will perform the action.) (refer to environment actions type 2).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4_sigmoid.png">

In [10]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action probabilities for the four actions. Print the observation and the action probabilities."""

environment = WumpusWorldEnvironment(observation_type='vector', action_type='multi_discrete')

observation = environment.reset()
# BEGIN_YOUR_CODE
print(observation)
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(36, 64)  # 5*5 from image dimension
        self.fc2 = nn.Linear(64, 4)
        

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        # using sigmoid activation function
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

 


net = Net()

environment_height = 6
environment_width = 6
coordinates_state_mapping = {}

for i in range(environment_height):
    for j in range(environment_width):
        coordinates_state_mapping[f'{np.asarray([j, i])}'] = i * environment_width + j

observationState= coordinates_state_mapping[f'{observation}']

observation_vector = np.zeros(36)
observation_vector[observation] = 1
observation_tensor = torch.tensor(observation_vector, dtype=torch.float32)
observation_tensor = observation_tensor.view(-1, 36)
output = net(observation_tensor)

print(output)

output_np_array = output.detach().numpy()
# action_probabilities = [1 if output_np_array[x] > 0.5 else 0 for x in range (len(output_np_array))]
action_probabilities = np.around(output_np_array)
print(action_probabilities)
print("New observation is", environment.step(np.ravel(action_probabilities)))

# END_YOUR_CODE


[0 0]
tensor([[ 0.2406, -0.1378,  0.6598,  0.2669]], grad_fn=<AddmmBackward0>)
[[ 0. -0.  1.  0.]]
New observation is (array([0, 1]), None, None, {})


## Observation Type - Vector (2.2), Action Type - Continuous

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 1 neuron. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an float in the range [-1, 1] determining the action which will be taken. (refer to environment actions type 3).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector and use the TanH activation function to get the output in the range [-1, 1].

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_1.png">

In [11]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action. Print the observation and the predicted action."""

environment = WumpusWorldEnvironment(observation_type='vector', action_type='continuous')
observation = environment.reset()

# BEGIN_YOUR_CODE

print(observation)
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(36, 64)  # 5*5 from image dimension
        self.fc2 = nn.Linear(64, 1)
        

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        #using tanh activation function
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

 


net = Net()

environment_height = 6
environment_width = 6
coordinates_state_mapping = {}

for i in range(environment_height):
    for j in range(environment_width):
        coordinates_state_mapping[f'{np.asarray([j, i])}'] = i * environment_width + j



observation_vector = np.zeros(36)
observation_vector[observation] = 1
observation_tensor = torch.tensor(observation_vector, dtype=torch.float32)
observation_tensor = observation_tensor.view(-1, 36)
output = net(observation_tensor)


output_np_array = output.detach().numpy()
action_value = np.ravel(output_np_array)[0]
print("Action value is", action_value)
print("New observation is", environment.step(action_value))

# END_YOUR_CODE

[0 0]
Action value is -0.011828424
New observation is (array([0, 0]), None, None, {})


c:\Users\abhil\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
